In [ ]:
%load_ext kamu

In [ ]:
%%local
import pandas as pd
import hvplot.pandas
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 1000)

In [ ]:
%import_dataset matches --alias matches
%import_dataset player-scores
%import_dataset player-scores-total

In [ ]:
%%sql -o scores
select * from `player-scores`

In [ ]:
%%local
scores.hvplot.box("score", by="name", height=500, width=800, title="Players score distribution")

In [ ]:
%%sql -o best_scores
select 
    name, 
    max(score_top) as score_top
from `player-scores-total`
group by name
order by 2 desc

In [ ]:
%%local
best_scores.hvplot.bar(x="name", y="score_top")

In [ ]:
from pyspark.sql.functions import col, udf
from gzip import compress

udf_compress = udf(compress)
matches_gz = matches.withColumn("replay_json_gz", udf_compress(col("replay_json")))
matches_gz.createOrReplaceTempView("matches_gz")

In [ ]:
%%sql
select event_time, match_id, replay_json_gz from matches_gz